In [ ]:
import cv2
import numpy as np

#데이터 정제하기

BLUE = 0
GREEN = 1
RED = 2

#특정한 색상의 모든 단어가 포함된 이미지 추출

def get_chars(image, color):
    other_1 = (color + 1) % 3
    other_2 = (color + 2) % 3

    c = image[:, :, other_1] == 255 #초록색 제거
    image[c] = [0, 0, 0]
    c = image[:, :, other_2] == 255 #빨간색 제거
    image[c] = [0, 0, 0]
    c = image[:, :, color] < 170 #초록색 + 빨간색 제거
    image[c] = [0, 0, 0]
    c = image[:, :, color] != 0 #파란색을 하얀색으로 만들기
    image[c] = [255, 255, 255]

    return image #파란색이 섞여 있거나 파란색만 남은 부분만 남기기(예시가 파란색이므로 다른색으로 하면 다른 결과가 나오게됨)

# 전체 이미지에서 왼쪽부터 단어별로 이미지를 추출
def extract_chars(image):
    chars = []
    colors = [BLUE, GREEN, RED]
    for color in colors:
        image_from_one_color = get_chars(image.copy(), color)
        image_gray = cv2.cvtColor(image_from_one_color, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(image_gray, 127, 255, 0)
        # RETR_EXTERNAL 옵션으로 숫자의 외각을 기준으로 분리
        contours, _ = cv2.findCOntours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # 추출된 이미지 크기가 50 이상인 경우만 실제 문자 데이터인 것으로 파악
            area = cv2.contourAREa(contour)
            if area > 50:
                x, y, width, height = cv2.boundingRect(contour)
                roi = image_gray[y:y + height, x:x + width]
                chars.append((x, roi))
    chars = sorted(chars, key = lambda char: char[0])
    return chars

# 특정한 이미지를 (20 * 20) 크기로 Scaling
def resize20(image):
    resized = cv2.resize(image, (20, 20))
    return resized.reshape(-1, 400).astype(np.float32)

In [2]:
import os
import cv2
import utils

# training_data 폴더 생성 및 그 내부에 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 폴더 생성
image = cv2.imread('image.png')
chars = uitls.extract_chars(image)

for char in chars:
    cv2.imshow('Image', char[1])
    input = cv2.waitkey(0)
    resized = cv2.resize(char[1], (20, 20))

    if input >= 48 and input <= 57:
        name = str(input - 48)
        file_count = len(next(os.walk('./training_data/' + name + '/'))[2])
        cv2.imwrite('./training_data/' + str(imput - 48) + '/' + str(file_count + 1) + '.png', resized)
        elif input == ord('a') or input == ord('b') or input == ord('c'):
            name = str(input - ord('a') + 10)
            file_count = len(next(os.walk('./training_data/' + name + '/'))[2])
            cv2.imwrite('./training_data/' + name + '/' + str(file_count + 1) + '.png', resized)

# KNN 모델 학습하기
file_names = list(range(0, 13))
train = []
train_lables = []

for file_name in file_names:
    path = './training_data/' + str(file_name) + '/'
    file_count = len(next(os.walk(path))[2])
    for i in range( 1, file_count + 1):
        img = cv2.inread(path + str(i) + '.png')
        gray = cv2.cvtColor(img, cv2.COLOR_BAYER_BG22GRAY)
        train.append(gray)
        train_labels.append(file_name)

x = np.array(train)
train = x[:, :].reshape(-1, 400).astype(np.float32)
train_labels = np.array(train_leables)[:, np.newaxis]

np.savez("trained.npz", traing = train, train_labels = train_labels)

SyntaxError: invalid syntax (<ipython-input-2-7bace9fb8ecd>, line 18)

In [ ]:
import numpy as np
import cv2
import utils

FILE_NAME = "trained.npz"

# 각 글자의 (1 * 400) 데이터와 정답 (0 ~ 9, +, - , *)
with np.load(FILE_NAME) as data:
    train = data['train']
    train_labels = data['train_labels']

knn = cv2.ml.KNEarest_create()
knn.train(train,cv2.ml.ROW_SAMPLE, train_labels)

def check(test, train, train_labels):
    # 가장 가까운 K개의 글자를 찾아, 어떤 숫자에 해당하는지 찾기(데이터 개수에 따라 조절)
    ret, result, neighbours, dist = knn.findNearest(test,  k = 1)
    return result

def get_result(file_name):
    image = cv2.imread(file_name)
    chars = utils.extract_chars(image)
    result_string = ""
    
    for char in chars:
        matched = check(utils.resize20(char[1]), train, train_labels)
        if matched < 10:
            result_string += str(int(matched))
            continue
        if matched == 10:
            matched = "+"
        elif matched == 11:
            matched = "-"
        elif matched == 12:
            matched = "*"
        result_string += matched
    
    

In [ ]:
# 수식 정제

import re

def remove_first_0(string):
    temp = []
    for i in string:
        if i == '+' or i == '-' or i == '*':
            temp.append(i)
    split = re.split('\*|\+|-', string)
    i = 0
    temp_count = 0
    result = ""
    for a in split:
        a = a.lstrip('0')
        if a == '':
            a = '0'
        result += a
        if i < len(split) - 1:
            result += temp[temp_count]
            temp_count = temp_count + 1
        i = i + 1
    return result
        